# Deep Learning - MCH2
Fachdozent: Martin Melchior     
Student: Manuel Schwarz   
HS23

Dieses Notebook bearbeitet die Mini-Challenge 2 des Moduls Deep Learning (del).   
Die Performance der Modelle wurde mit **wandb.ai** aufgezeichnet und kann [hier](https://wandb.ai/manuel-schwarz/del-mc2/workspace?workspace=user-manuel-schwarz) eingesehen werden.  

<div class="alert alert-block alert-info">
<b>Aufgabenstellung:</b> Eine Blaue Box beschreibt die Aufgabe aus der Aufgabenstellung 'SGDS_DEL_MC1.pdf' 
</div>

<div class="alert alert-block alert-success">
<b>Antworte:</b> Eine Grüne Box beschreibt die Bearbeitung / Reflektion der Aufgabenstellung
</div>

In [ ]:
import torch

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device: ', device)

In [ ]:
import os
import copy
import time
import torch
import wandb
import random
import spacy  # conda install -c conda-forge spacy + python -m spacy download en_core_web_sm
import torchvision
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
from PIL import Image
from tqdm import tqdm 
from datetime import datetime
from spacy.symbols import ORTH
from collections import Counter
from torch.optim import lr_scheduler
from torch.utils.data import Dataset
from sklearn.model_selection import KFold
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision import datasets, models, transforms

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device: ', device)

# sound
import time
import winsound
import datetime

### Aufbau Modellierung und Daten

<div class="alert alert-block alert-info">

Überlege Dir, welche Modell-Architektur Sinn machen könnte. Mindestens zwei Modell-Varianten sollen aufgebaut werden, die miteinander verglichen werden sollen.

</div>

<div class="alert alert-block alert-success">
Für die del-MC2 Challenge wird das Modell vom Paper Vinyals et al `Show and Tell: A Neural Image Caption Generator` nachgebaut. Das Paper entwickelte ein Modell welches für Bilder eine Bildbeschreibung erstellt. Für die verwendeten Daten wird das `Flickr 8k` Datenset verwendet. 
</div>

### Hilfsfunktionen

In [ ]:
def set_seed(seed=42):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
set_seed()

### Daten Flickr 8k lesen

In [ ]:
images_folder = './data/Images'
captions_file = './data/captions.txt'

In [ ]:
pd_captions = pd.read_csv('./data/captions.txt', sep='\t', header=None)
pd_captions.columns = ['full_caption']
pd_captions[['image_name', 'caption']] = pd_captions['full_caption'].str.split(',', n=1, expand=True)
pd_captions.to_csv('./data/pd_captions.csv', index=False)
pd_captions.drop('full_caption', axis=1, inplace=True)
pd_captions.head(10)

Im `caption.txt` File ist der Bildnamen und die Bildbeschreibung (caption) hinterlegt. Pro Bild stehen fünf Captions zur Verfügung.

In [ ]:
image_id = 5

example_image_path = f'{images_folder}/{pd_captions.image_name[image_id]}'
example_caption1 = pd_captions.caption[image_id+0]
example_caption2 = pd_captions.caption[image_id+1]
example_caption3 = pd_captions.caption[image_id+2]
example_caption4 = pd_captions.caption[image_id+3]
example_caption5 = pd_captions.caption[image_id+4]
image = Image.open(example_image_path)

plt.imshow(image)
plt.title(f'{example_caption1} \n {example_caption2} \n {example_caption3} \n {example_caption4} \n {example_caption5}')
plt.axis('off') 
plt.show()

### Untersuchen der Bilddaten

In [ ]:
images_folder_path = './data/Images/'

resolutions = []
for image_filename in os.listdir(images_folder_path):
    if image_filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        image_path = os.path.join(images_folder_path, image_filename)
        with Image.open(image_path) as img:
            resolutions.append(img.size)

In [ ]:
dimension_labels = [f"{w}x{h}" for w, h in resolutions]
resolution_counts = Counter(dimension_labels)
sorted_resolution_counts = dict(resolution_counts.most_common(50))

plt.figure(figsize=(10, 3))
plt.bar(sorted_resolution_counts.keys(), sorted_resolution_counts.values(), color='skyblue')

plt.title('Verteilung der Bildauflösungen von Flickr 8k (Top 50)')
plt.xlabel('Dimension (b x w)')
plt.ylabel('Anzahl Bilder')
plt.xticks(rotation=90)
plt.show()

In [ ]:
widths, heights = zip(*resolutions)

plt.figure(figsize=(10, 3))

plt.subplot(1, 2, 1)
plt.boxplot(widths, patch_artist=True, boxprops=dict(facecolor='grey', color='black'),
            whiskerprops=dict(color='black'), capprops=dict(color='black'),
            medianprops=dict(color='red'))
plt.title('Verteilung der Bildbreiten')
plt.ylabel('Pixel')
plt.xticks([1], ['b'])

plt.subplot(1, 2, 2) 
plt.boxplot(heights, patch_artist=True, boxprops=dict(facecolor='grey', color='black'),
            whiskerprops=dict(color='black'), capprops=dict(color='black'),
            medianprops=dict(color='red'))
plt.title('Verteilung der Bildhöhen')
plt.ylabel('Pixel')
plt.xticks([1], ['h'])

plt.tight_layout()
plt.show()

### Untersuchen der Bildbeschreibungen

Textdaten werden folgend in Tokens konvergiert:
[doku torchtext](http://man.hubwiz.com/docset/torchtext.docset/Contents/Resources/Documents/index.html)


In [ ]:
# !pip show torchtext
# Version: 0.6.0

In [ ]:
spacy_en = spacy.load('en_core_web_sm')

special_cases = [("<start>", [{ORTH: "<start>"}]), ("<end>", [{ORTH: "<end>"}]), ("<pad>", [{ORTH: "<pad>"}])]
for case in special_cases:
    spacy_en.tokenizer.add_special_case(*case)

def tokenize_en(caption, lower_text=False):
    if lower_text:
        return [tok.text.lower() for tok in spacy_en.tokenizer(caption)]
    else:
        return [tok.text for tok in spacy_en.tokenizer(caption)]    

print(f'Test tokenize: {example_caption1}')
tokens = tokenize_en(example_caption1)
tokens

In [ ]:
# liste mit allen vorhandenen Tokens 
token_series = pd_captions['caption'].apply(tokenize_en).explode()
count_token = token_series.value_counts()
count_token.index.name = 'token'
count_token = count_token.reset_index()
# count_token

In [ ]:
num_token = 50
plt.figure(figsize = (10, 4))
plt.bar(count_token.head(num_token).token, count_token.head(num_token).caption, color='skyblue')
plt.title(f'Total Tokens: {len(count_token)} Tokens, dargestellt {num_token} Tokens')
plt.xticks(rotation=90)
plt.xlabel('Token')
plt.ylabel('Vorkommen')
plt.show()

In [ ]:
def tokenize_en_len(caption, lower_text=False):
    if lower_text:
        return len([tok.text.lower() for tok in spacy_en.tokenizer(caption)])
    else:
        return len([tok.text for tok in spacy_en.tokenizer(caption)])

token_series = pd_captions['caption'].apply(tokenize_en_len)
# token_series

In [ ]:
plt.figure(figsize = (10, 3))
token_mean = token_series.mean()
token_std = token_series.std() / 2
plt.hist(token_series, color='skyblue', bins=40)
plt.axvline(token_mean, color='red', alpha=0.6, label=f'mean {token_mean:0.2f}')
plt.axvspan(token_mean-token_std, token_mean+token_std, color='grey', alpha=0.2, label=f'std {token_std:0.2f}')
plt.suptitle('Verteilung der Anzahl Tokens je Caption')
plt.title(f'min bei: {token_series.min()}, max bei: {token_series.max()}', fontsize=8)
plt.xlabel('Anzahl Tokens')
plt.ylabel('Vorkommen')
plt.legend()
plt.show()

### Bearbeitung der Captions für Modelling

- Im Paper werden Tokens die weniger als fünf mal vorkommen entfernt
- Im Paper werden `start` und `end` Token eingeführt, diese sollen dem Modell helfen zu erkennen, wann der Beginn der Generierung einer Beschreibung ist und wann sie beendet werden sollte.

In [ ]:
min_num_token = 5
count_token_filtered = count_token[count_token.caption >= min_num_token]
print(f'Total Tokens {len(count_token)}')
print(f'Anzahl Tokens die mehr als {min_num_token} vorkommen: {len(count_token_filtered)}')

In [ ]:
START_TOKEN = "<start>"
END_TOKEN = "<end>"

pd_caption_mod = pd_captions.copy()
pd_caption_mod['caption'] = pd_caption_mod['caption'].apply(lambda x: f"{START_TOKEN} {x} {END_TOKEN}")
pd_captions.to_csv('./data/pd_captions_mod.csv', index=False)

print(pd_captions['caption'][0])
print(pd_captions['caption'][1])
print(pd_captions['caption'][2])
print()
print(pd_caption_mod['caption'][0])
print(pd_caption_mod['caption'][1])
print(pd_caption_mod['caption'][2])


### Aufteilung in Trainings- und Testdaten

In [ ]:
print(f'Anzahl Captions: {len(pd_caption_mod)}')
unique_images = pd_caption_mod.image_name.unique()
print(f'Anzahl Bilder: {len(unique_images)}')

unique_images = list(pd_caption_mod.image_name.unique())
train_images = random.sample(unique_images, k=int(len(unique_images) * 0.8))
test_images = list(set(unique_images) - set(train_images))

print(f'Länge Trainingsset: {len(train_images)}')
print(f'Länge Testset: {len(test_images)}')

pd_train_set = pd_caption_mod[pd_caption_mod.image_name.isin(train_images)]
pd_test_set = pd_caption_mod[pd_caption_mod.image_name.isin(test_images)]
pd_train_set.to_csv('./data/train_captions.csv', index=False)
pd_test_set.to_csv('./data/test_captions.csv', index=False)

print(f'Länge Trainingsset: {len(pd_train_set)}')
print(f'Länge Testset: {len(pd_test_set)}')


In [ ]:
train_set = pd.read_csv('./data/train_captions.csv')
test_set = pd.read_csv('./data/test_captions.csv')

train_set.head(10)

In [ ]:
image_id = 5

example_image_path = f'{images_folder}/{train_set.image_name[image_id]}'
example_caption1 = train_set.caption[image_id+0]
example_caption2 = train_set.caption[image_id+1]
example_caption3 = train_set.caption[image_id+2]
example_caption4 = train_set.caption[image_id+3]
example_caption5 = train_set.caption[image_id+4]
image = Image.open(example_image_path)

plt.imshow(image)
plt.title(f'{example_caption1} \n {example_caption2} \n {example_caption3} \n {example_caption4} \n {example_caption5}')
plt.axis('off') 
plt.show()

### Bearbeitung der Captions für Modelling (Trainingsset)
- caption in matrix ablegen, gleiche länge der captions

In [ ]:
token_series = train_set['caption'].apply(tokenize_en_len)
print(f'maximale caption länge: {token_series.max()} Tokens')

In [ ]:
def build_vocab(tokenized_captions, min_freq):
    # Zähle die Häufigkeit der Tokens in allen Captions
    token_counts = Counter(token for caption in tokenized_captions for token in caption)

    # Erstelle das Vokabular nur mit Tokens, die min_freq oder mehr Mal vorkommen
    vocab = {
        "<pad>": 0,
        "<start>": 1,
        "<end>": 2
    }
    token_id = 3
    for token, count in token_counts.items():
        if count >= min_freq:
            vocab[token] = token_id
            token_id += 1
    return vocab

def create_matrices_from_captions(train_set):
    captions = train_set['caption']
    tokenized_captions = [tokenize_en(caption, lower_text=True) for caption in captions]

    # Erstellen des Vokabular mit einer Mindesthäufigkeit von x
    vocab = build_vocab(tokenized_captions, min_freq=5)

    # Vokabular verwenden, um Ihre Captions in Indizes umzuwandeln
    indexed_captions = [[vocab.get(token, vocab["<pad>"]) for token in caption] for caption in tokenized_captions]

    caption_tensors = [torch.tensor(caption) for caption in indexed_captions]

    # Bestimmen der maximale Länge für das Padding
    max_length = max(len(caption) for caption in caption_tensors)

    # Padding hinzufügen, damit alle Captions die gleiche Länge haben
    padded_captions = pad_sequence(caption_tensors, batch_first=True, padding_value=vocab["<pad>"])

    return padded_captions, vocab


In [ ]:
padded_captions, vocab = create_matrices_from_captions(train_set)
print(f'Matrix dim: {padded_captions.shape}')
print(f'Länge Wörterbuch: {len(vocab)}')
padded_captions[:5]


### Erstellen des Dataloaders

In [ ]:
class FlickrDataset(Dataset):
    def __init__(self, csv_file_name, root_dir, transform=None):
        self.captions_frame = pd.read_csv(csv_file_name)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.captions_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.captions_frame.iloc[idx, 0])
        image = Image.open(img_name)
        caption = self.captions_frame.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)
        return image, caption
    




In [ ]:
transformations = transforms.Compose([
    # transforms.Resize((256, 256)),
    # transforms.RandomCrop(224),
    transforms.ToTensor()
])

train_set = FlickrDataset(
    csv_file_name='./data/train_captions.csv',
    root_dir='./data/Images',
    transform=transformations
)

test_set = FlickrDataset(
    csv_file_name='./data/test_captions.csv',
    root_dir='./data/Images',
    transform=transformations
)

train_dataloader = DataLoader(train_set, batch_size=1, shuffle=True, num_workers=0)
test_dataloader = DataLoader(test_set, batch_size=1, shuffle=False, num_workers=0)

In [ ]:
# Test Dataloader
for i_batch, (image, caption) in enumerate(train_dataloader):
    # print(f'{i_batch}: {image.shape}, {caption}')
    to_pil = transforms.ToPILImage()
    img_pil = to_pil(image.squeeze(0))
    plt.imshow(img_pil)
    plt.title(caption)
    plt.axis('off')
    plt.show()
    break